# Issues / Improvements with OL

## Preface
This document aims at collecting ideas for possible improvements of OL and differences between the current benchmark concerning vectortiles rendering which is the [vanilla mapbox](http://vectortiles.geo.admin.ch/gl-styles/ch.swisstopo.leichte-basiskarte.vt/v006/) viewer. The "model under test" is [vanilla OL](https://mf-geoadmin4.int.bgdi.ch/ol-minimalist/index.html) using `ol-mapbox-style` library to interpret mapbox style files and generate `VectorTilesLayer`s (see figure below). The style files are slightly modified to use only a single source host (`vectortiles100.geo.admin.ch`) instead of a range of five hosts (performance improvement on browsers supporting `http/2`). 

There are a few open [issues](https://github.com/geoadmin/mf-geoadmin3/issues?q=is%3Aissue+is%3Aopen+%5BMVT%5D+milestone%3A2019-04-29) that need to be solved before the public go-live of the [testviewer](test.map.geo.admin.ch) on 29.4.2019. 
1. [[MVT] link on Re3 to test viewer 1st prio](https://github.com/geoadmin/mf-geoadmin3/issues/4864)
2. [[MVT] caching not enabled 1st prio bug](https://github.com/geoadmin/mf-geoadmin3/issues/4861)
3. [[MVT] Client side zoom for vector tiles 1st prio MVT enhancement](https://github.com/geoadmin/mf-geoadmin3/issues/4860)
4. [[MVT] OL differences with Mapbox rendering 1st prio MVT bug](https://github.com/geoadmin/mf-geoadmin3/issues/4752) ("bundle" issue, collects all issues concerning styling with ol-mapbox-style)
5. [[MVT] labels hide location circle on zoom 4 1st prio MVT bug enhancement](https://github.com/geoadmin/mf-geoadmin3/issues/4724)
(Note that the issue titles describe the problems from a user experince point of view and are not necessarily related to the technical reason for the behaviour (e.g. caching)).

The issues related to the visual representation (1., 4., 5.) are most likely solvable till end of April, the performance related issues (2., 3.) are out of reach to be solved until 29.4.2019 since they are directly related to core parts of OL. This documents explains the current limitations and possible solutions for the open issues in more detail.

Conceptually there exist(ed) two different approaches to integrate a vector tiles layer using mapbox vector tiles in an OL map. In the first one - let's call it *ol native* - OL delegates only the task of layer creation and styling to a third party library called [ol-mapbox-style]() but takes care of layer rendering itself (using `CanvasVectorTileLayerRenderer`). In the second one - *ol + mapbox* - the mapbox-gl renderer is wrapped in an OL layer and hence treated like a "normal" OL layer, since (after a refactoring [PR #8922](https://github.com/openlayers/openlayers/pull/8922)) each layer is responsible for its rendering himself. 

<!--![native ol vs. ol + mapbox-gl](https://ltboc.infra.bgdi.ch/notebooks/mvt/ol-native-vs-ol+mapbox.svg)-->
![native ol vs. ol + mapbox-gl](assets/ol-native-vs-ol+mapbox.png)

*native ol vs. ol + mapbox-gl*

A POC of this approach is available [here](https://codepen.io/pakb/pen/pBjGOP). It seems however that essentials parts have been removed from OL (maybe related to removal of WebGL parts) so this approach doesn't work with the current OL-`master` and is therefore not further evaluated in this document. Nevertheless, it seems a compelling idea to use mapbox-gl for rendering mapbox vectortiles as an OL layer. The possible limitations of this approach have yet to be evaluated though.

## TOC

- [Performance](#Performance)
    - [Timing of re-rendering and loading new tiles](#Timing-of-loading-new-tiles)    
    - [Rendering](#Rendering)
    - [Animation](#Animation)
    - [Caching](#Caching)
- [Styling](#Styling)
- [Documentation](#Documentation)
- [Roadmap](#Roadmap)

---

## Performance

We are comparing the performance of vanilla OL with vanilla MB rendering the vectortiles with the style produced by swisstopo Karto department. Performance improvements that could result from modifying / optimizing the style are explicitly noten taken into account here, MB renderer is able to cope with the style and amount of data in a performant way, so this is the "gold standard". 

### Timing of loading new tiles
Work has already been done lately to improve the user experience during zomm/load cycles, namely with [PR #9101](https://github.com/openlayers/openlayers/pull/9101) which allows to load tiles also during animation/interaction phases, [PR #9128](https://github.com/openlayers/openlayers/pull/9128) which reduces render cycles of VT layers not to block animation/interaction and [PR #9152](https://github.com/openlayers/openlayers/pull/9152) which made further improvements / fine tuning to  balance computing power between rendering and animation/interaction.

However, when recording performance of a mouse scroll zoom action it becomes apparent that the timing of loading new data is only when the zoom-animation has finished. 

![vanilla ol](assets/ol-late-tile-loading.png)
*vanilla ol*

Comparing this to mapbox reveals that in MB, loading newer data is fired right after the zoom event has occured. Thus, tile loading has finished once animation has finished and rendering of the new data can immediately start. 

- **In what ways can this behaviour further be improved in OL?**
- **how is it determined, when new data has to be loaded?** Can this be made predictive (e.g. panning left immediately start loading new tiles on the right of the map)

### Rendering
The overall rendering performance can't keep up with mapbox in general. As mentioned, this refers to vanilla OL and vanilla OL rendering the same vector tiles source from swisstopo. Since MB is optimized for rendering vector tiles, the bar is set high and many improvements have already been made over the last months.

#### Webworkers
As stated in [1], "currently all costly operations are done on the main thread, and data preparation/parsing takes at least as much time as rendering". Work has been done in OL to offload tile preparation work to webworkers. In a first step ([PR #8926](https://github.com/openlayers/openlayers/pull/8926)) execution planning and rendering has been separated, followed by [PR #9008](https://github.com/openlayers/openlayers/pull/9008) which decouples render instruction creation from rendering
and [PR #9058](https://github.com/openlayers/openlayers/pull/9058) which moves all tile loading logic from the VectorImageTile to the source.

- **What is the status here?**
- **Is the [*`OffscreenCanvas`*](https://developer.mozilla.org/en-US/docs/Web/API/OffscreenCanvas#Browser_compatibility)([4]) feature necessary to make progress here? (as mentioned in 'comments' in [1])** 
Browser-Support for offscreencanvas is still very limited and considering that we'll have to support older browser (certainly EDGE 17) for a while, are there other options? Workers currently seem to be used only in the Raster source. 

#### WebGL
In [1] (deliverables 1 and 2) WebGL is mentioned several times as full-fledged renderer, however with [PR #8951](https://github.com/openlayers/openlayers/pull/8951) most support for WebGL was dropped. 

- **What was the reason for this and could this be reconsidered?** 

Performance analysis shows that MB is extensively using GPU, it seems that it can be beneficial.


### Animation

A major zooming issue was recently resolved with [PR #9357](https://github.com/openlayers/openlayers/pull/9357) by rendering vector tiles at the view resolution instead of fixed steps which resulted an majory differences in what is displayed at which zoom level compared to MB. 
Nevertheless, since `vectorial` zoom / pan animation was removed with [PR #8923](https://github.com/openlayers/openlayers/pull/8923) other visual side effects can be observed:
- feature size doesn't change smootly between zoom levels: features are enlarged too much by zooming the rastered image, subsequent rerendering reduces the feature size to the correct value. Vector zooming would allow for a smooth transition between feature sizes (this behaviour can be best observed at roundabouts).
![roundabout zooming](assets/roundabout.gif)
- "fading in" of features when zooming is not working as intended (best observed in forest areas), MB zoom smoothly changes the forest color whereas OL zoom animation uses the "old" color and then abruptly changes to the new color.

The visual effect of the described behaviours can be slightly reduced by tweaking zoom steps and animation times which has been done in the [vanilla ol](https://mf-geoadmin4.int.bgdi.ch/ol-minimalist/index.html). Thusly, the effect of `hybrid` rendering can be reduced.
Zooming by mouse wheel is currently too coarse (zooming step of one per wheel 'tick'), can be reduced to 0.33 (as in Mapbox) by reducing the `MAX_DELTA` constant (in `interaction/MouseWheelZoom.js:17`
```interaction/MouseWheelZoom.js:17
/**
 * Maximum mouse wheel delta.
 * @type {number}
 */
const MAX_DELTA = 0.3;
```

Furthermore, there is a difference in mouse wheel zooming and trackpad zooming. Trackpad shows a smoother zooming behaviour (more frequent re-rendering) than mouse wheel zooming. Performance recording shows that trackpad zooming consists of numerous small steps with intermediate re-rendering while mouse-wheel zooming is one long animation and subsequent re-rendering.

- **Why was 'vector' mode dropped and could this be reconsidered?**

### Caching

Caching (specially cache size calculation) was already improved wiht [PR #9318](https://github.com/openlayers/openlayers/pull/9318). It seems however that caching strategy of tiles is still not optimal. This is most apparent when zooming out of a region that was previously already displayed at a lower zoom level. Checking network traffic reveals that tiles are (partly?) loaded again that have been previously loaded (although most of them from browser cache). Again, MB doesn't show this behaviour, when zooming in and out again at the exact same point, no tiles are reloaded.

### Style Optimizations

As stated in [issue #136](https://github.com/openlayers/ol-mapbox-style/issues/136) OL cannot declutter on map level. In order to be able to use two tile sources (OSM and swisstopo) and have a correct layering, ol-mapbox-style has to create five layers as shown in the first figure. Three of them only contain labels. This is not very performant, so a possible optimization would be to create a combined source and apply decluttering already there. This would result in just a single `VectorTileLayer` and a possibly considerable performance boost. Another approach to handle this in the library is currently being worked on (see [issue #136](https://github.com/openlayers/ol-mapbox-style/issues/136) for updates).

---

## Styling

As mentioned in the preface, in the *"ol-native"* approach an additional library is needed to interpret and translate mapbox style definition. 
[ol-mapbox-style](https://github.com/openlayers/ol-mapbox-style) does a decent job already. However, there are still major and minor [differences](https://github.com/geoadmin/mf-geoadmin3/issues/4752) that have to be resolved to provide an acceptable representation of mapbox styled vector tiles in OL.

- **The advancement/resolution of these issues seems to be blocked due to financial issues. How can we proceed in this matter?**

---

## Documentation

The existing API documentation is good but "higher" level docs of concepts, architecture (e.g. call tree), rendering process are missing (or not available to us).

- **is such doc existing?**
*-> Nope*

---

## Roadmap

In order to plan the next development steps at swisstopo it is important to know the development pipeline of OL. Many of the mentioned goals in [2] and [1] have been reached, many are yet to be addressed. As mentioned in the preface, there are two fundamentally different ways of having vector tiles rendered in OL, *'ol-native'* (which requires ol-mapbox-style) and *'ol + mapbox'*. Brief answers from the telco with c2c (Belo and Gravin) are added in *italics*

### *ol-native* (plus ol-mapbox-style)
This is merely a summary of points mentioned above, mainly concerning performance. What is the status of

- **using webworkers?**
    *-> A lot of preliminary work has been done (see details above, mainly to allow serialization of rendering tasks). Substantial performance improvements are expected from OffscreenCanvas (rendering a whole canvas can be offloaded to workers), but they still have very limited browser support. Further development has been stalled atm, since without offscreencanvas, perf. improvements are limited and/or would require an enormous effort.*

- **using WebGL?**
    *-> The WebGL code in OL was removed because it was not well tested and not really maintained by the community. Competence in WebGL at c2c the OL community is currently very limited. Integrating WebGL code into a framework is fairly complex and puts high demands on knowhow and money (Having a WebGL renderer developed properly would require at least ~ 3Mio. CHF, rough estimation by c2c). The experimental WebGL code in OL caused a lot of work in the past with limited outcome, it's simply too much work. Therefore, canvas-only rendering and still is more developed and maintained and has seen major improvements over the last six months. Nevertheless, certain refactoring was done to simplify the integration of renderers using WebGL, mainly by offloading the rendering duty to the layer (such each layer can have a different renderer). The heatmap renderer is an example of using WebGL to render simple primitives (points), but is far away from being capable of rendering complex geometries like polygons, fonts etc. typically found in vectortiles, it's just "a black hole". But the opinion in the community about a WebGL renderer in OL is not uniform, some like AHocevar are in favour of a pure canvas-only OL framework, others like TSchaub are much more in favour of WebGL.*

- **ol-mapbox-style?** currently there seem to be financial issues blocking further development. Plans to integrate it into openlayers?

### *ol + mapbox*
Following up on the question whether delegating the rendering of a VT layer to mapbox-gl (encapsulated e.g. in a `MapboxLayer` as in the examples [here](https://170-4723248-gh.circle-artifacts.com/0/examples/mapbox-layer.html) (from [PR #8931](https://github.com/openlayers/openlayers/pull/8931)) and [here](https://codepen.io/pakb/pen/pBjGOP)) is a viable way to go as mentioned as a goal in the [OL 6 call for funding](https://github.com/openlayers/openlayers/wiki/OpenLayers-6:-call-for-funding):

- **general limitations with regard to other OL functionality?**
    *-> A general POC of using mapbox-gl as renderer for an OL layer was done during the code sprint in Nov '18. Hereby, mapbox-gl provides the WebGL part. This POC is however not production ready. Mainly the performance implications on various browsers and platforms have to be carefully tested since the two renderers would need to be synchronized for every frame. Also the implications on functional/application level have to be carefully analysed since two renderers are loaded, which means the two libraries need to be synchronized. For certain functionality like `forEachPixelOnLayer`, sync functions would have to be developed. Additionally, the current POC has issues like the 'glitches' when zooming out* 

- **effort to have this working with current `master` again**
    *-> This should work with current `master`, examples will be sent to Pascal*

---


## Resources

[[1]](https://docs.google.com/spreadsheets/d/1VZA57VBC37PWPP1vofD5HBXld2XgdsE92z1xdBN41bI/edit#gid=0) RE4_Crowdfunding_Acceptance_Protocol 

[[2]](https://docs.google.com/document/d/1CZndl2HZ8kuXAb7nHzMyCshT1zCBkKCJBRb6mSzy9Ok/edit#heading=h.3ec7yuydoshz) ol6 vt code sprint (goal and preparations)

[[3]](https://www.html5rocks.com/en/tutorials/canvas/performance/) about HTML 5 Canvas performance

[[4]](https://developers.google.com/web/updates/2018/08/offscreen-canvas) `OffscreenCanvas()`